# 📊 Mastering SQLite3 with Python
### A Comprehensive Guide from Zero to Advanced

This notebook serves as a complete sandbox for learning **SQLite3**. Unlike other databases, SQLite doesn't require a server—it's just a file on your disk, making it perfect for apps, data science, and prototyping.

---
**What we will build:** A Digital Library Management System.

## 1. Setup & Architecture
SQLite is unique because the entire database is a single file.



First, let's import the library and check our version.

In [14]:
import sqlite3
import pandas as pd

print(f"SQLite3 version: {sqlite3.sqlite_version}")
print(f"Python sqlite3 module version: {sqlite3.version}")

SQLite3 version: 3.37.2
Python sqlite3 module version: 2.6.0


/tmp/ipython-input-3153500092.py:5: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(f"Python sqlite3 module version: {sqlite3.version}")


## 2. The Connection & The Cursor
- **Connection:** The bridge to your database file.
- **Cursor:** The "pointer" used to execute commands and fetch results.

In [15]:
# Connect to a database (creates 'library.db' if it doesn't exist)
connection = sqlite3.connect('library.db')

# Create a cursor object
cursor = connection.cursor()

print("Database created and connected successfully.")

Database created and connected successfully.


## 3. Creating Tables with Constraints
We'll create a table for `books`. Notice the use of `PRIMARY KEY`, `NOT NULL`, and `CHECK` constraints to ensure data quality.

In [16]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    published_year INTEGER,
    price REAL CHECK(price >= 0)
)
''')

connection.commit()
print("Table 'books' is ready.")

Table 'books' is ready.


## 4. Inserting Data (The Secure Way)
**Warning:** Never use f-strings or string formatting for SQL queries (risk of SQL Injection). Use the `?` placeholder.

In [17]:
# Single Insert
cursor.execute("INSERT INTO books (title, author, published_year, price) VALUES (?, ?, ?, ?)",
               ('The Great Gatsby', 'F. Scott Fitzgerald', 1925, 12.99))

# Bulk Insert (Efficient)
more_books = [
    ('1984', 'George Orwell', 1949, 15.50),
    ('To Kill a Mockingbird', 'Harper Lee', 1960, 10.00),
    ('The Hobbit', 'J.R.R. Tolkien', 1937, 20.00),
    ('Brave New World', 'Aldous Huxley', 1932, 14.25)
]

cursor.executemany("INSERT INTO books (title, author, published_year, price) VALUES (?, ?, ?, ?)", more_books)

connection.commit()
print(f"{cursor.rowcount} total rows inserted.")

4 total rows inserted.


## 5. Querying & Data Analysis
Let's fetch data using different methods.

In [18]:
print("--- All Books ---")
cursor.execute("SELECT * FROM books")
for row in cursor.fetchall():
    print(row)

print("\n--- Books Newer than 1940 (Sorted by Year) ---")
cursor.execute("SELECT title, published_year FROM books WHERE published_year > 1940 ORDER BY published_year ASC")
print(cursor.fetchall())

--- All Books ---
(1, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 15.717900000000004)
(2, '1984', 'George Orwell', 1949, 18.755000000000003)
(3, 'To Kill a Mockingbird', 'Harper Lee', 1960, 12.100000000000001)
(4, 'The Hobbit', 'J.R.R. Tolkien', 1937, 24.200000000000003)
(5, 'Brave New World', 'Aldous Huxley', 1932, 17.242500000000003)
(6, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925, 12.99)
(7, '1984', 'George Orwell', 1949, 15.5)
(8, 'To Kill a Mockingbird', 'Harper Lee', 1960, 10.0)
(9, 'The Hobbit', 'J.R.R. Tolkien', 1937, 20.0)
(10, 'Brave New World', 'Aldous Huxley', 1932, 14.25)

--- Books Newer than 1940 (Sorted by Year) ---
[('1984', 1949), ('1984', 1949), ('To Kill a Mockingbird', 1960), ('To Kill a Mockingbird', 1960)]


## 6. Integration with Pandas
For Data Science, you often want your SQL results in a DataFrame.

In [8]:
df = pd.read_sql_query("SELECT author, COUNT(*) as book_count, AVG(price) as avg_price FROM books GROUP BY author", connection)
df

,author,book_count,avg_price
0,Aldous Huxley,1,14.25
1,F. Scott Fitzgerald,1,12.99
2,George Orwell,1,15.50
3,Harper Lee,1,10.00
4,J.R.R. Tolkien,1,20.00


## 7. Advanced: Transactions (ACID Compliance)
Transactions ensure that if one part of a multi-step process fails, the whole thing rolls back. This prevents partial data corruption.

In [11]:
try:
    # Start a transaction
    connection.execute("BEGIN TRANSACTION")

    cursor.execute("UPDATE books SET price = price * 1.1") # 10% inflation

    # Simulating an error (e.g., trying to insert NULL into a NOT NULL column)
    # cursor.execute("INSERT INTO books (title) VALUES (NULL)")

    connection.commit()
    print("Prices updated successfully.")
except sqlite3.Error as e:
    print(f"An error occurred: {e}. Rolling back changes.")
    connection.rollback()

Prices updated successfully.


## 8. Cleanup
Always close your connection to free up memory and file locks.

In [12]:
connection.close()
print("Database connection closed.")

Database connection closed.
